In [ ]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional

In [ ]:
import os

my_path = "/content/drive/MyDrive/Online Courses/PW Skills/NLP"
os.chdir(my_path)
print(os.listdir())

['Fake News Classification', 'model.h5', 'LSTM.ipynb']


In [ ]:
import pandas as pd

df = pd.read_csv("Fake News Classification/train.csv")
print(df.shape)
df.head()

(20800, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# Checking anf dropping Nan values
print(df.isna().sum())
df = df.dropna()
print(df.isna().sum())
print(df.shape)

id           0
title      558
author    1957
text        39
label        0
dtype: int64
id        0
title     0
author    0
text      0
label     0
dtype: int64
(18285, 5)


In [ ]:
X = df.drop(labels='label', axis=1)
y = df['label']
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


### Data Preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
messages = X.copy()
messages.reset_index(inplace=True)
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


1. Clean the text and reduce the non stop words to its stem


In [ ]:
corpus = []
ps = PorterStemmer()
for ind in range(0, len(messages)):
  sent = re.sub("[^a-zA-Z]", " ", messages['title'][ind])
  sent = sent.lower()
  words = sent.split()

  # If word is not a stop word, stem it and add to list
  stem_word = [ps.stem(word) for word in words if word not in nltk.corpus.stopwords.words('english')]
  sent = " ".join(stem_word)
  corpus.append(sent)

In [ ]:
print(type(corpus), len(corpus))

<class 'list'> 18285


2. Embed the corpus using one hot encoding
- **one_hot()**   It assigns each word in the corpus a unique integer (from 1 to n, where n is the vocab_size), representing a word's position in the vocabulary. The integer is chosen using a hashing function.

In [ ]:
vocab_size = 5000
onehot_repr = [one_hot(input_text=words, n=vocab_size)for words in corpus]

3. Pad the sequences

In [ ]:
max_len = 30 # Assumed maximum no. of words in any sentence
embedded_doc = pad_sequences(sequences=onehot_repr, maxlen=max_len, padding='post')
embedded_doc = np.array(embedded_doc)
y = np.array(y)
embedded_doc.shape

(18285, 30)

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(embedded_doc, y, test_size=0.2, shuffle=True, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(14628, 30) (3657, 30) (14628,) (3657,)


### Model Building

In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=40, input_length=max_len),
    LSTM(units=100),
    Dense(1, activation='sigmoid')
])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
len(X_train)/64

228.5625

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.7458 - loss: 0.4374 - val_accuracy: 0.9171 - val_loss: 0.1811
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.9426 - loss: 0.1461 - val_accuracy: 0.9232 - val_loss: 0.2017
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.9630 - loss: 0.1044 - val_accuracy: 0.9182 - val_loss: 0.2162
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9717 - loss: 0.0843 - val_accuracy: 0.9223 - val_loss: 0.2750
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - accuracy: 0.9780 - loss: 0.0646 - val_accuracy: 0.9057 - val_loss: 0.2428
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.9782 - loss: 0.0623 - val_accuracy: 0.9185 - val_loss: 0.2756
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.9837 - loss: 0.0467 - val_accuracy: 0.9182 - val_loss: 0.2915
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.9898 - loss: 0.0330 - val_a

In [ ]:
model.save('model.h5')

In [ ]:
y_pred = (model.predict(X_test)>0.5).astype('int64')
y_pred.shape

115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


(3657, 1)

### Checking model accuracy
_____
- true negatives = C_{0,0}
- false negatives = C_{1,0}
- true positives = C_{1,1}
- false positives = C_{0,1}

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(y_true=y_test, y_pred=y_pred))
print(accuracy_score(y_true=y_test, y_pred=y_pred))

[[1928  148]
 [ 168 1413]]
0.9135903746240087


## Bidirectional LSTM

In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=40, input_length=max_len),
    Bidirectional(LSTM(units=100)),
    Dense(1, activation='sigmoid')
])

print(model.summary())

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

y_pred = (model.predict(X_test)>0.5).astype('int64')
print(y_pred.shape)

print(confusion_matrix(y_true=y_test, y_pred=y_pred))
print(accuracy_score(y_true=y_test, y_pred=y_pred))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.7853 - loss: 0.4283 - val_accuracy: 0.9243 - val_loss: 0.1816
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 18s 55ms/step - accuracy: 0.9488 - loss: 0.1294 - val_accuracy: 0.9281 - val_loss: 0.1809
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.9646 - loss: 0.0991 - val_accuracy: 0.9281 - val_loss: 0.1815
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.9791 - loss: 0.0694 - val_accuracy: 0.9243 - val_loss: 0.2429
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 20s 55ms/step - accuracy: 0.9853 - loss: 0.0483 - val_accuracy: 0.9202 - val_loss: 0.2581
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - accuracy: 0.9892 - loss: 0.0359 - val_accuracy: 0.9204 - val_loss: 0.3035
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.9926 - loss: 0.0237 - val_accuracy: 0.9180 - val_loss: 0.3366
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.9941 - loss: 0.02